### Merging data

In [1]:
# import modules

import numpy as np
import pandas as pd

In [3]:
# import zipcodes
zipcode = pd.read_csv("data/zipfips.csv")
zipcode.head()

,Unnamed: 0,zipcode,latitude,longitude,city,state,county,fips
0,0,501,40.922326,-72.637078,Holtsville,NY,Suffolk,36103.0
1,1,544,40.922326,-72.637078,Holtsville,NY,Suffolk,NaN
2,2,601,18.165273,-66.722583,Adjuntas,PR,Adjuntas,72001.0
3,3,601,18.165273,-66.722583,Adjuntas,PR,Adjuntas,72113.0
4,4,602,18.393103,-67.180953,Aguada,PR,Aguada,72003.0


In [4]:
# function for data files

def imp_acsdf(num, samp ):
    dfname = "df" + num 
    print("Dataframe Name is: %s" % dfname)
    if samp == False:
        dfname = pd.read_csv("./data/ACS_16_5YR_DP" + num + "_with_ann.csv",skiprows=[1], low_memory=False)
    else:
        dfname = pd.read_csv("./data/ACS_16_5YR_DP" + num + "_with_ann.csv", nrows = 1000, skiprows=[1] )
    print (dfname.shape)
    return dfname

In [5]:
# function for meta files

def imp_acsmeta(num ):

    metaname = "meta" + num

    print("MetaData Name is: %s" % metaname)
    metaname = pd.read_csv("./data/ACS_16_5YR_DP" + num + "_metadata.csv", header=None, index_col = 0 )
    print(metaname.shape)
    return metaname

In [6]:
dict_df = {}
dict_meta={}

for i in range(4):
    numstr = "0" + str(i+2)

    key_namedf = "df" + numstr     

    dict_df[key_namedf] = imp_acsdf(num = numstr, samp = False) # change to false to import all data

    key_namemeta = "meta" + numstr    

    dict_meta[key_namemeta] = imp_acsmeta(num = numstr)
       

dict_df.keys()

Dataframe Name is: df02
(32989, 611)
MetaData Name is: meta02
(611, 1)
Dataframe Name is: df03
(33120, 551)
MetaData Name is: meta03
(551, 1)
Dataframe Name is: df04
(33120, 575)
MetaData Name is: meta04
(575, 1)
Dataframe Name is: df05
(33120, 339)
MetaData Name is: meta05
(339, 1)


dict_keys(['df02', 'df03', 'df04', 'df05'])

In [7]:
# add df and meta to dict
df02 = dict_df['df02']
df03 = dict_df['df03']
df04 = dict_df['df04']
df05 = dict_df['df05']
meta02 = dict_meta['meta02'].to_dict()[1]
meta03 = dict_meta['meta03'].to_dict()[1]
meta04 = dict_meta['meta04'].to_dict()[1]
meta05 = dict_meta['meta05'].to_dict()[1]

In [8]:
# function to extact estimates
def extract_Est(df):
    df = df[df.columns.drop(list(df.filter(regex='HC02')))]
    df = df[df.columns.drop(list(df.filter(regex='HC03')))]
    df = df[df.columns.drop(list(df.filter(regex='HC04')))]
    return df

In [9]:
# Extract only the estimates in df which all start with 'HCO1'
df02 = extract_Est(df02)
df03 = extract_Est(df03)
df04 = extract_Est(df04)
df05 = extract_Est(df05)

In [10]:
# Function to extract meaningful categories from variable names
def extract_catdf(df, dname, dno):
    df = pd.DataFrame.from_dict(df,  orient='index')
    df = df.rename_axis('v').reset_index().rename(columns={0:'detail'})
    df =df.iloc[3:] 
    df['type'] = df.detail.str.split(';').str[0]
    df['cat0'] = dname 
    df['cat1'] = df.detail.str.split(';').str[1].str.split('-',1).str[0].str.strip()
    df['cat2'] = df.detail.str.split(';').str[1].str.split('-',1).str[1].str.strip()
    df['vname'] = dno + df.v
    
    return df

In [11]:
# name categories extracted from variable names
m02 = extract_catdf(meta02, "Social", "02")
m03 = extract_catdf(meta03, "Economic","03")
m04 = extract_catdf(meta04, "Housing","04")
m05 = extract_catdf(meta05 , "Demographic","05")

In [12]:
# concatenate meta categories
meta = pd.concat([m02, m03, m04, m05], ignore_index=True)

In [13]:
print(meta.type.unique())
print(meta.cat0.unique())
print(meta.cat1.unique())
print(meta.cat2.unique())
print(meta.cat3.unique())

['Estimate' 'Margin of Error' 'Percent' 'Percent Margin of Error']
['Social' 'Economic' 'Housing' 'Demographic']
['HOUSEHOLDS BY TYPE' 'RELATIONSHIP' 'MARITAL STATUS' 'FERTILITY'
 'GRANDPARENTS' 'SCHOOL ENROLLMENT' 'EDUCATIONAL ATTAINMENT'
 'VETERAN STATUS'
 'DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION'
 'RESIDENCE 1 YEAR AGO' 'PLACE OF BIRTH' 'U.S. CITIZENSHIP STATUS'
 'YEAR OF ENTRY' 'WORLD REGION OF BIRTH OF FOREIGN BORN'
 'LANGUAGE SPOKEN AT HOME' 'ANCESTRY' 'COMPUTERS AND INTERNET USE'
 'EMPLOYMENT STATUS' 'COMMUTING TO WORK' 'OCCUPATION' 'INDUSTRY'
 'CLASS OF WORKER' 'INCOME AND BENEFITS (IN 2016 INFLATION'
 'HEALTH INSURANCE COVERAGE'
 'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL'
 'HOUSING OCCUPANCY' 'UNITS IN STRUCTURE' 'YEAR STRUCTURE BUILT' 'ROOMS'
 'BEDROOMS' 'HOUSING TENURE' 'YEAR HOUSEHOLDER MOVED INTO UNIT'
 'VEHICLES AVAILABLE' 'HOUSE HEATING FUEL' 'SELECTED CHARACTERISTICS'
 'OCCUPANTS PER ROOM' 

AttributeError: 'DataFrame' object has no attribute 'cat3'

In [15]:
meta = meta[meta.type == 'Estimate']

In [16]:
def rename_cols(df, num):
    cols = df.columns[~df.columns.str.contains('GEO')]
    df.rename(columns = dict(zip(cols, num + cols )), inplace=True)
    return df

In [17]:
df02 = rename_cols(df02,'02')
df03 = rename_cols(df03,'03')
df04 = rename_cols(df04,'04')
df05 = rename_cols(df05,'05')

In [18]:
# merge data files with zipcodes
print(zipcode.shape)
df = pd.merge(left=zipcode, right=df02, right_on='GEO.id2', left_on='zipcode', how = 'left')
print(df.shape)
df = pd.merge(left=df, right=df03, right_on='GEO.id2', left_on='zipcode', how = 'left')
print(df.shape)
df = pd.merge(left=df, right=df04, right_on='GEO.id2', left_on='zipcode', how = 'left')
print(df.shape)
df = pd.merge(left=df, right=df05, right_on='GEO.id2', left_on='zipcode', how = 'left')
print(df.shape)

(57163, 8)
(57163, 163)
(57163, 303)
(57163, 449)
(57163, 536)


In [19]:
df.head()

,Unnamed: 0,zipcode,latitude,longitude,city,state,county,fips,GEO.id_x,GEO.id2_x,...,05HC01_VC97,05HC01_VC98,05HC01_VC99,05HC01_VC100,05HC01_VC101,05HC01_VC102,05HC01_VC104,05HC01_VC108,05HC01_VC109,05HC01_VC110
0,0,501,40.922326,-72.637078,Holtsville,NY,Suffolk,36103.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,544,40.922326,-72.637078,Holtsville,NY,Suffolk,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,601,18.165273,-66.722583,Adjuntas,PR,Adjuntas,72001.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,7290.0,13646.0,6573.0,7073.0
3,3,601,18.165273,-66.722583,Adjuntas,PR,Adjuntas,72113.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,7290.0,13646.0,6573.0,7073.0
4,4,602,18.393103,-67.180953,Aguada,PR,Aguada,72003.0,NaN,NaN,...,59.0,0.0,31.0,960.0,32.0,928.0,17312.0,31048.0,15156.0,15892.0


In [20]:
meta.head()

,v,detail,type,cat0,cat1,cat2,vname
0,HC01_VC03,Estimate; HOUSEHOLDS BY TYPE - Total households,Estimate,Social,HOUSEHOLDS BY TYPE,Total households,02HC01_VC03
4,HC01_VC04,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families),02HC01_VC04
8,HC01_VC05,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC05
12,HC01_VC06,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC06
16,HC01_VC07,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC07


In [21]:
# save df and meta to csv
df.to_csv("./data/df.csv")
meta.to_csv("./data/meta.csv")

### Cleaning data

In [22]:
# function to group data
def group_data(df, geo = 'country'):
    if geo == 'country':
        df
    elif geo == 'state':
        df = df.groupby("state").sum()
    elif geo == 'county':
        df = df.groupby(["state", "county"]).sum()
    elif geo == 'city':
        df = df.groupby(["state", "county", "city"]).sum()
    return df

In [23]:
grouped_data = group_data(df, geo = 'city')
grouped_data.head()

Unnamed: 0  zipcode   latitude   longitude  \
state county         city                                                     
AK    Aleutians East Akutan           56926    99553  55.430594 -162.558130   
                     Cold Bay         56941    99571  55.650875 -160.654522   
                     False Pass       56952    99583  55.430594 -162.558130   
                     King Cove        56972    99612  55.430594 -162.558130   
                     Sand Point       57014    99661  55.350356 -160.485506   

                                   fips  GEO.id2_x  02HC01_VC03  02HC01_VC04  \
state county         city                                                      
AK    Aleutians East Akutan      2013.0    99553.0         54.0         27.0   
                     Cold Bay    2013.0    99571.0         52.0         25.0   
                     False Pass  2013.0    99583.0         31.0         15.0   
                     King Cove   2013.0    99612.0        246.0        169.0   
                     Sand Point  2013.0    99661.0        405.0        263.0   

                                 02HC01_VC05  02HC01_VC06      ...       \
state county         city                                      ...        
AK    Aleutians East Akutan              8.0         18.0      ...        
                     Cold Bay            4.0         23.0      ...        
                     False Pass          6.0          6.0      ...        
                     King Cove          77.0        113.0      ...        
                     Sand Point        140.0        138.0      ...        

                                 05HC01_VC97  05HC01_VC98  05HC01_VC99  \
state county         city                                                
AK    Aleutians East Akutan            297.0         22.0          0.0   
                     Cold Bay            0.0          0.0          0.0   
                     False Pass          4.0          0.0          0.0   
                     King Cove         379.0          4.0          0.0   
                     Sand Point        292.0         10.0          0.0   

                                 05HC01_VC100  05HC01_VC101  05HC01_VC102  \
state county         city                                                   
AK    Aleutians East Akutan              13.0           0.0          13.0   
                     Cold Bay             3.0           0.0           3.0   
                     False Pass           0.0           0.0           0.0   
                     King Cove           49.0           0.0          49.0   
                     Sand Point          86.0           0.0          86.0   

                                 05HC01_VC104  05HC01_VC108  05HC01_VC109  \
state county         city                                                   
AK    Aleutians East Akutan              62.0         367.0         260.0   
                     Cold Bay           106.0         102.0          56.0   
                     False Pass          47.0          44.0          28.0   
                     King Cove          295.0         671.0         439.0   
                     Sand Point         472.0         782.0         491.0   

                                 05HC01_VC110  
state county         city                      
AK    Aleutians East Akutan             107.0  
                     Cold Bay            46.0  
                     False Pass          16.0  
                     King Cove          232.0  
                     Sand Point         291.0  

[5 rows x 463 columns]

In [24]:
meta['level1'] = meta.cat2.str.split('-',1).str[0].str.strip()
meta['level2'] = meta.cat2.str.split('-',1).str[1].str.strip()
meta['level3'] = meta.level2.str.split('-',1).str[1].str.strip()

In [25]:
meta.head()

,v,detail,type,cat0,cat1,cat2,vname,level1,level2,level3
0,HC01_VC03,Estimate; HOUSEHOLDS BY TYPE - Total households,Estimate,Social,HOUSEHOLDS BY TYPE,Total households,02HC01_VC03,Total households,NaN,NaN
4,HC01_VC04,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families),02HC01_VC04,Total households,Family households (families),NaN
8,HC01_VC05,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC05,Total households,Family households (families) - With own childr...,With own children of the householder under 18 ...
12,HC01_VC06,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC06,Total households,Family households (families) - Married-couple ...,Married-couple family
16,HC01_VC07,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC07,Total households,Family households (families) - Married-couple ...,Married-couple family - With own children of t...


In [26]:
df.describe()

,Unnamed: 0,zipcode,latitude,longitude,fips,GEO.id2_x,02HC01_VC03,02HC01_VC04,02HC01_VC05,02HC01_VC06,...,05HC01_VC97,05HC01_VC98,05HC01_VC99,05HC01_VC100,05HC01_VC101,05HC01_VC102,05HC01_VC104,05HC01_VC108,05HC01_VC109,05HC01_VC110
count,57163.000000,57163.000000,56436.000000,56436.000000,53467.000000,46965.000000,46965.000000,46965.000000,46965.000000,46965.000000,...,47257.000000,47257.000000,47257.000000,47257.000000,47257.000000,47257.000000,47257.000000,47257.000000,47257.000000,47257.000000
mean,28581.000000,49268.873275,38.509356,-90.433289,29998.783960,49672.259023,3436.436325,2286.384329,980.683041,1690.494262,...,414.075989,12.139260,18.159976,200.061218,7.695177,192.366041,3990.069111,6723.684153,3256.909537,3466.774615
std,16501.681056,26640.371876,5.302350,15.269060,15518.847622,25677.868428,4936.433948,3278.631393,1532.780859,2408.296240,...,1783.291993,135.337163,99.203722,436.743299,31.716741,423.807535,5547.377237,9496.737857,4550.462611,4967.558927
min,0.000000,501.000000,-7.209975,-176.787412,1001.000000,1001.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,14290.500000,28063.000000,35.124321,-96.671008,18065.000000,29135.000000,333.000000,230.000000,78.000000,182.000000,...,0.000000,0.000000,0.000000,4.000000,0.000000,3.000000,440.000000,668.000000,336.000000,328.000000
50%,28581.000000,48867.000000,39.109550,-88.055804,29173.000000,49769.000000,1130.000000,780.000000,291.000000,607.000000,...,7.000000,0.000000,0.000000,36.000000,0.000000,35.000000,1421.000000,2273.000000,1135.000000,1136.000000
75%,42871.500000,70649.000000,41.873852,-80.734879,42063.000000,68929.000000,4412.000000,2987.000000,1205.000000,2176.000000,...,109.000000,0.000000,4.000000,194.000000,0.000000,188.000000,5263.000000,8905.000000,4350.000000,4548.000000
max,57162.000000,99950.000000,70.494693,166.410291,78030.000000,99929.000000,42783.000000,27649.000000,16999.000000,22827.000000,...,60407.000000,15389.000000,11889.000000,15479.000000,1792.000000,15305.000000,48196.000000,78545.000000,35405.000000,43140.000000


In [27]:
df.select_dtypes(include=['object']).columns

Index(['city', 'state', 'county', 'GEO.id_x', 'GEO.display-label_x',
       '02HC01_VC21', '02HC01_VC22', '02HC01_VC52', '02HC01_VC53',
       '02HC01_VC54', '02HC01_VC55', '02HC01_VC56', '02HC01_VC57',
       '02HC01_VC58', '02HC01_VC95', '02HC01_VC96', '02HC01_VC216',
       '02HC01_VC217', '02HC01_VC218', 'GEO.id_y', 'GEO.display-label_y',
       '03HC01_VC12', '03HC01_VC36', '03HC01_VC85', '03HC01_VC86',
       '03HC01_VC90', '03HC01_VC92', '03HC01_VC94', '03HC01_VC98',
       '03HC01_VC100', '03HC01_VC114', '03HC01_VC115', '03HC01_VC118',
       '03HC01_VC121', '03HC01_VC122', '03HC01_VC124', '03HC01_VC125',
       '03HC01_VC126', '03HC01_VC161', '03HC01_VC162', '03HC01_VC163',
       '03HC01_VC164', '03HC01_VC165', '03HC01_VC166', '03HC01_VC167',
       '03HC01_VC168', '03HC01_VC169', '03HC01_VC171', '03HC01_VC172',
       '03HC01_VC173', '03HC01_VC174', '03HC01_VC175', '03HC01_VC176',
       '03HC01_VC177', '03HC01_VC178', '03HC01_VC179', '03HC01_VC180',
       'GEO.id_x', 'GEO.

In [28]:
columns = ['02HC01_VC21', '02HC01_VC22', '02HC01_VC52', '02HC01_VC53',
       '02HC01_VC54', '02HC01_VC55', '02HC01_VC56', '02HC01_VC57',
       '02HC01_VC58', '02HC01_VC95', '02HC01_VC96', '02HC01_VC216',
       '02HC01_VC217', '02HC01_VC218', '03HC01_VC12', '03HC01_VC36',
       '03HC01_VC85', '03HC01_VC86', '03HC01_VC90', '03HC01_VC92',
       '03HC01_VC94', '03HC01_VC98', '03HC01_VC100', '03HC01_VC114',
       '03HC01_VC115', '03HC01_VC118', '03HC01_VC121', '03HC01_VC122',
       '03HC01_VC124', '03HC01_VC125', '03HC01_VC126', '03HC01_VC161',
       '03HC01_VC162', '03HC01_VC163', '03HC01_VC164', '03HC01_VC165',
       '03HC01_VC166', '03HC01_VC167', '03HC01_VC168', '03HC01_VC169',
       '03HC01_VC171', '03HC01_VC172', '03HC01_VC173', '03HC01_VC174',
       '03HC01_VC175', '03HC01_VC176', '03HC01_VC177', '03HC01_VC178',
       '03HC01_VC179', '03HC01_VC180', '04HC01_VC08', '04HC01_VC09',
       '04HC01_VC50', '04HC01_VC69', '04HC01_VC70', '04HC01_VC106',
       '04HC01_VC108', '04HC01_VC128', '04HC01_VC146', '04HC01_VC155',
       '04HC01_VC191', '05HC01_VC23']

In [29]:
df[columns] = df[columns].apply(pd.to_numeric, errors='coerce')

In [30]:
# see column types
df.dtypes

Unnamed: 0               int64
zipcode                  int64
latitude               float64
longitude              float64
city                    object
state                   object
county                  object
fips                   float64
GEO.id_x                object
GEO.id2_x              float64
GEO.display-label_x     object
02HC01_VC03            float64
02HC01_VC04            float64
02HC01_VC05            float64
02HC01_VC06            float64
02HC01_VC07            float64
02HC01_VC08            float64
02HC01_VC09            float64
02HC01_VC10            float64
02HC01_VC11            float64
02HC01_VC12            float64
02HC01_VC13            float64
02HC01_VC14            float64
02HC01_VC17            float64
02HC01_VC18            float64
02HC01_VC21            float64
02HC01_VC22            float64
02HC01_VC26            float64
02HC01_VC27            float64
02HC01_VC28            float64
                        ...   
05HC01_VC72            float64
05HC01_V

In [31]:
df.select_dtypes(include=['object']).columns

Index(['city', 'state', 'county', 'GEO.id_x', 'GEO.display-label_x',
       'GEO.id_y', 'GEO.display-label_y', 'GEO.id_x', 'GEO.display-label_x',
       'GEO.id_y', 'GEO.display-label_y'],
      dtype='object')

In [32]:
# get info on data
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57163 entries, 0 to 57162
Columns: 536 entries, Unnamed: 0 to 05HC01_VC110
dtypes: float64(523), int64(2), object(11)
memory usage: 234.2+ MB
None


In [33]:
# # DataFrame method to check for missing values in a column
# # The .all() method returns True if all values are True
# # if using it on a DataFrame, you need to chain another .all() 
# # method so that you return only one True or False value.
# assert pd.notnull(df['GEO.id_x']).all().all()
# assert pd.notnull(df['HC01_VC150']).all().all()
# assert (df.HC01_VC150 >=0).all().all()

In [34]:
df.tail(20)

,Unnamed: 0,zipcode,latitude,longitude,city,state,county,fips,GEO.id_x,GEO.id2_x,...,05HC01_VC97,05HC01_VC98,05HC01_VC99,05HC01_VC100,05HC01_VC101,05HC01_VC102,05HC01_VC104,05HC01_VC108,05HC01_VC109,05HC01_VC110
57143,57143,99832,58.331798,-135.181256,Pelican,AK,Skagway Hoonah Angoon,2105.0,8600000US99832,99832.0,...,0.0,0.0,0.0,19.0,0.0,19.0,126.0,90.0,38.0,52.0
57144,57144,99833,56.767056,-132.931950,Petersburg,AK,Wrangell Petersburg,2195.0,8600000US99833,99833.0,...,142.0,20.0,2.0,245.0,12.0,233.0,1759.0,2341.0,1289.0,1052.0
57145,57145,99835,57.080818,-135.318348,Sitka,AK,Sitka,2220.0,8600000US99835,99835.0,...,605.0,33.0,0.0,873.0,44.0,829.0,4100.0,6601.0,3350.0,3251.0
57146,57146,99836,56.535950,-132.827948,Port Alexander,AK,Wrangell Petersburg,2195.0,8600000US99836,99836.0,...,0.0,0.0,0.0,6.0,0.0,6.0,65.0,31.0,14.0,17.0
57147,57147,99840,59.456753,-135.314852,Skagway,AK,Skagway Hoonah Angoon,2230.0,8600000US99840,99840.0,...,76.0,0.0,2.0,45.0,4.0,41.0,639.0,823.0,470.0,353.0
57148,57148,99841,57.798249,-135.285061,Tenakee Springs,AK,Skagway Hoonah Angoon,2105.0,8600000US99841,99841.0,...,0.0,0.0,0.0,0.0,0.0,0.0,205.0,143.0,80.0,63.0
57149,57149,99850,58.383480,-134.197786,Juneau,AK,Juneau,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57150,57150,99901,55.400674,-131.674090,Ketchikan,AK,Ketchikan Gateway,2130.0,8600000US99901,99901.0,...,1049.0,10.0,74.0,1259.0,0.0,1259.0,6254.0,10184.0,5327.0,4857.0
57151,57151,99903,55.517921,-132.003244,Meyers Chuck,AK,Prince Wales Ketchikan,2275.0,8600000US99903,99903.0,...,0.0,0.0,0.0,0.0,0.0,0.0,23.0,16.0,8.0,8.0
57152,57152,99918,55.517921,-132.003244,Coffman Cove,AK,Prince Wales Ketchikan,2198.0,8600000US99918,99918.0,...,0.0,0.0,0.0,5.0,0.0,5.0,233.0,170.0,97.0,73.0


In [35]:
# # save df and meta to csv
# df.to_csv("./data/df1.csv")
# meta.to_csv("./data/meta1.csv")

In [36]:
df.head()

,Unnamed: 0,zipcode,latitude,longitude,city,state,county,fips,GEO.id_x,GEO.id2_x,...,05HC01_VC97,05HC01_VC98,05HC01_VC99,05HC01_VC100,05HC01_VC101,05HC01_VC102,05HC01_VC104,05HC01_VC108,05HC01_VC109,05HC01_VC110
0,0,501,40.922326,-72.637078,Holtsville,NY,Suffolk,36103.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,544,40.922326,-72.637078,Holtsville,NY,Suffolk,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,601,18.165273,-66.722583,Adjuntas,PR,Adjuntas,72001.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,7290.0,13646.0,6573.0,7073.0
3,3,601,18.165273,-66.722583,Adjuntas,PR,Adjuntas,72113.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,7290.0,13646.0,6573.0,7073.0
4,4,602,18.393103,-67.180953,Aguada,PR,Aguada,72003.0,NaN,NaN,...,59.0,0.0,31.0,960.0,32.0,928.0,17312.0,31048.0,15156.0,15892.0


In [37]:
meta.head()

,v,detail,type,cat0,cat1,cat2,vname,level1,level2,level3
0,HC01_VC03,Estimate; HOUSEHOLDS BY TYPE - Total households,Estimate,Social,HOUSEHOLDS BY TYPE,Total households,02HC01_VC03,Total households,NaN,NaN
4,HC01_VC04,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families),02HC01_VC04,Total households,Family households (families),NaN
8,HC01_VC05,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC05,Total households,Family households (families) - With own childr...,With own children of the householder under 18 ...
12,HC01_VC06,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC06,Total households,Family households (families) - Married-couple ...,Married-couple family
16,HC01_VC07,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC07,Total households,Family households (families) - Married-couple ...,Married-couple family - With own children of t...


In [38]:
social = meta[meta['cat0'] == 'Social']
total_household = social[social['level1'] == 'Total households']

In [39]:
total_household

,v,detail,type,cat0,cat1,cat2,vname,level1,level2,level3
0,HC01_VC03,Estimate; HOUSEHOLDS BY TYPE - Total households,Estimate,Social,HOUSEHOLDS BY TYPE,Total households,02HC01_VC03,Total households,NaN,NaN
4,HC01_VC04,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families),02HC01_VC04,Total households,Family households (families),NaN
8,HC01_VC05,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC05,Total households,Family households (families) - With own childr...,With own children of the householder under 18 ...
12,HC01_VC06,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC06,Total households,Family households (families) - Married-couple ...,Married-couple family
16,HC01_VC07,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC07,Total households,Family households (families) - Married-couple ...,Married-couple family - With own children of t...
20,HC01_VC08,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC08,Total households,Family households (families) - Male householde...,"Male householder, no wife present, family"
24,HC01_VC09,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC09,Total households,Family households (families) - Male householde...,"Male householder, no wife present, family - Wi..."
28,HC01_VC10,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC10,Total households,Family households (families) - Female househol...,"Female householder, no husband present, family"
32,HC01_VC11,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC11,Total households,Family households (families) - Female househol...,"Female householder, no husband present, family..."
36,HC01_VC12,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Nonfamily households,02HC01_VC12,Total households,Nonfamily households,NaN


In [40]:
total_household.shape

(15, 10)

In [41]:
total_household = total_household[: -3]

In [42]:
total_household

,v,detail,type,cat0,cat1,cat2,vname,level1,level2,level3
0,HC01_VC03,Estimate; HOUSEHOLDS BY TYPE - Total households,Estimate,Social,HOUSEHOLDS BY TYPE,Total households,02HC01_VC03,Total households,NaN,NaN
4,HC01_VC04,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families),02HC01_VC04,Total households,Family households (families),NaN
8,HC01_VC05,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC05,Total households,Family households (families) - With own childr...,With own children of the householder under 18 ...
12,HC01_VC06,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC06,Total households,Family households (families) - Married-couple ...,Married-couple family
16,HC01_VC07,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC07,Total households,Family households (families) - Married-couple ...,Married-couple family - With own children of t...
20,HC01_VC08,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC08,Total households,Family households (families) - Male householde...,"Male householder, no wife present, family"
24,HC01_VC09,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC09,Total households,Family households (families) - Male householde...,"Male householder, no wife present, family - Wi..."
28,HC01_VC10,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC10,Total households,Family households (families) - Female househol...,"Female householder, no husband present, family"
32,HC01_VC11,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Family households (families...,02HC01_VC11,Total households,Family households (families) - Female househol...,"Female householder, no husband present, family..."
36,HC01_VC12,Estimate; HOUSEHOLDS BY TYPE - Total household...,Estimate,Social,HOUSEHOLDS BY TYPE,Total households - Nonfamily households,02HC01_VC12,Total households,Nonfamily households,NaN


In [43]:
household_list = total_household["vname"].tolist()

In [44]:
household_list

['02HC01_VC03',
 '02HC01_VC04',
 '02HC01_VC05',
 '02HC01_VC06',
 '02HC01_VC07',
 '02HC01_VC08',
 '02HC01_VC09',
 '02HC01_VC10',
 '02HC01_VC11',
 '02HC01_VC12',
 '02HC01_VC13',
 '02HC01_VC14']

In [45]:
new_cols = ['zipcode', 'latitude', 'longitude', 'city', 'state', 'county', 'fips', '02HC01_VC03', '02HC01_VC04', '02HC01_VC05']

In [46]:
df_th = df[new_cols].fillna(0)
df_th = df_th[(df_th != 0).all(1)]

In [47]:
df_th.head()

,zipcode,latitude,longitude,city,state,county,fips,02HC01_VC03,02HC01_VC04,02HC01_VC05
356,1001,42.140549,-72.788661,Agawam,MA,Hampden,25013.0,7190.0,3943.0,1406.0
357,1002,42.367092,-72.464571,Amherst,MA,Hampshire,25011.0,9561.0,4626.0,2068.0
358,1002,42.367092,-72.464571,Amherst,MA,Hampshire,25015.0,9561.0,4626.0,2068.0
361,1005,42.329160,-72.139465,Barre,MA,Worcester,25027.0,1840.0,1150.0,462.0
362,1007,42.280267,-72.402056,Belchertown,MA,Hampshire,25015.0,5611.0,4010.0,1882.0


In [48]:
# save df to csv
df_th.to_csv("./data/df_th.csv")

In [49]:
df_th.iloc[0:20]

,zipcode,latitude,longitude,city,state,county,fips,02HC01_VC03,02HC01_VC04,02HC01_VC05
356,1001,42.140549,-72.788661,Agawam,MA,Hampden,25013.0,7190.0,3943.0,1406.0
357,1002,42.367092,-72.464571,Amherst,MA,Hampshire,25011.0,9561.0,4626.0,2068.0
358,1002,42.367092,-72.464571,Amherst,MA,Hampshire,25015.0,9561.0,4626.0,2068.0
361,1005,42.329160,-72.139465,Barre,MA,Worcester,25027.0,1840.0,1150.0,462.0
362,1007,42.280267,-72.402056,Belchertown,MA,Hampshire,25015.0,5611.0,4010.0,1882.0
363,1008,42.177833,-72.958359,Blandford,MA,Hampden,25003.0,530.0,366.0,108.0
364,1008,42.177833,-72.958359,Blandford,MA,Hampden,25013.0,530.0,366.0,108.0
365,1009,42.206092,-72.340486,Bondsville,MA,Hampden,25013.0,106.0,80.0,12.0
366,1010,42.108585,-72.204480,Brimfield,MA,Hampden,25013.0,1494.0,1026.0,369.0
367,1010,42.108585,-72.204480,Brimfield,MA,Hampden,25027.0,1494.0,1026.0,369.0


In [50]:
household_list1 = total_household["cat2"].tolist()
household_list1

['Total households',
 'Total households - Family households (families)',
 'Total households - Family households (families) - With own children of the householder under 18 years',
 'Total households - Family households (families) - Married-couple family',
 'Total households - Family households (families) - Married-couple family - With own children of the householder under 18 years',
 'Total households - Family households (families) - Male householder, no wife present, family',
 'Total households - Family households (families) - Male householder, no wife present, family - With own children of the householder under 18 years',
 'Total households - Family households (families) - Female householder, no husband present, family',
 'Total households - Family households (families) - Female householder, no husband present, family - With own children of the householder under 18 years',
 'Total households - Nonfamily households',
 'Total households - Nonfamily households - Householder living alone'